In [1]:
from langchain_core.prompts import PromptTemplate

from configs.agent import PROMPT_TEMPLATE

prompt = PromptTemplate.from_template(PROMPT_TEMPLATE)

In [2]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from utils.agent import load_reader_model
from scripts.tools import HappyFaceTool, ConsultancyTool

llm = load_reader_model()
tools = [HappyFaceTool(), ConsultancyTool()]

agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [3]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description(tools)
tool_names = ', '.join([tool.name for tool in tools])
print(rendered_tools)
print(tool_names)

HappyFaceRagSearchTool: Use this tool whenever you're responding to requests related to factual details of the
    HappyFace service platform
HappyFaceConsultantTool: Use this tool when addressing queries within the context of offering consultation, guidance, or 
    assistance to an individual actively seeking solutions for encountered personal problems or challenges
HappyFaceRagSearchTool, HappyFaceConsultantTool


In [4]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []


def get_agent_output(query: str) -> str:
    result = agent_executor.invoke({
        "input": query,
        "chat_history": chat_history,
        "tools": rendered_tools,
        "tool_names": tool_names
    })
    return result["output"]


def append_latest_conv_to_chat_history(query: str, output: str) -> None:
    latest_conv = [
        HumanMessage(content=query),
        AIMessage(content=output)
    ]
    chat_history.append(latest_conv)

In [ ]:
from utils.consultant import generate_summarized_profile

personal_profile = {
    "name": "Smith",
    "address": "USA",
    "dob": "2001-06-20",
    "gender": "male",
    "salary": 6000,
    "hiddenDiseases": ["Hepatitis"],
    "family": {
        "numMembers": 12,
        "monthlyIncome": 300000,
        "monthlyExpenses": 900000,
        "category": "nucleus"
    }
}
emotion_engagement_profile = {
    "name": "Smith",
    "gender": "male",
    "emotionalProfile": {
        "mostlyEngagingFacialEmotion": {
            "emotion": "fear",
            "avgArousal (percentage)": 90,
            "avgValence (percentage)": -60
        },
        "mostlyEngagingSentimentalEmotion": {
            "emotion": "anger",
            "avgArousal (percentage)": 69,
            "avgValence (percentage)": -100
        }
    }
}
query = '''
    Provide a personal consultation for an individual named Smith by utilizing HappyFace Consultancy Tool.
    
    Smith's profile data is shown below. 
    
    Personal Profile: {personal_profile}
    
    Emotion Engagement Profile: {emotion_engagement_profile}
    
    Here is the recent chat history which has already built between smith and consultant.
    
    Consultant Chat History: {consultant_chat_history}
'''.format(
    personal_profile=generate_summarized_profile(personal_profile),
    emotion_engagement_profile=generate_summarized_profile(emotion_engagement_profile),
    consultant_chat_history=[]
)
# print(query)
output = get_agent_output(query)
append_latest_conv_to_chat_history(query, output)
print(output)



> Entering new AgentExecutor chain...

Invoking: `HappyFaceConsultantTool` with `{'personal_profile': 'Smith is a male individual residing in the USA, born on June 20, 2001, making him 21 years old. His annual salary is 6,000, and he has been diagnosed with Hepatitis. In his nucleus family, there are twelve members, with a substantial monthly income of 300,000 but high monthly expenses of 900,000. The family might face financial challenges due to the significant difference between their income and expenses, which could require careful financial management and planning.', 'emotion_engagement_profile': "Smith is a male individual who exhibits predominantly negative emotions in his emotional profile. His most engaging facial emotion is 'fear' with very high arousal (90) and a very high negative valence (-60), suggesting intense feelings of anxiety or dread. Additionally, his engaging sentimental emotion is 'anger' with high arousal (69) and an extremely negative valence (-100), indicati